In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import Batch


In [5]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2) # 2 classes (causal, non-causal)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [6]:
# Example data (difference of mean and difference of std for edges)
edge_features_1 = torch.tensor([
    [0.2, 0.1], # edge 0 -> 1
    [-0.3, 0.2], # edge 1 -> 2
    [0.1, -0.2], # edge 2 -> 0
], dtype=torch.float)

# Edge index (source -> target)
edge_index_1 = torch.tensor([
    [0, 1, 2],
    [1, 2, 0],
], dtype=torch.long)

edge_labels_1 = torch.tensor([1, 0, 1], dtype=torch.long)

# Example data (difference of mean and difference of std for edges)
edge_features_2 = torch.tensor([
    [0.2, 0.1], # edge 0 -> 1
    [-0.3, 0.2], # edge 1 -> 2
    [0.1, -0.2], # edge 2 -> 0
], dtype=torch.float)

# Edge index (source -> target)
edge_index_2 = torch.tensor([
    [0, 1, 2],
    [1, 2, 0],
], dtype=torch.long)

edge_labels_2 = torch.tensor([1, 0, 1], dtype=torch.long)

graphs = [Data(x=edge_features_1, edge_index=edge_index_1, y=edge_labels_1),
          Data(x=edge_features_2, edge_index=edge_index_2, y=edge_labels_2)
          ]

train_data = Batch.from_data_list(graphs)

In [7]:
# Define model and optimizer
model = GNN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = F.cross_entropy(out, train_data.y)
    loss.backward()
    optimizer.step()


In [8]:
# New edge features for evaluation
eval_edge_features_1 = torch.tensor([
    [0.3, 0.1],  # edge 0 -> 1
    [-0.4, 0.2], # edge 1 -> 2
    [0.15, -0.2],# edge 2 -> 0
], dtype=torch.float)

# Edge index for evaluation (same as training, but can be different)
eval_edge_index_1 = torch.tensor([
    [0, 1, 2],
    [1, 2, 0],
], dtype=torch.long)

# New edge features for evaluation
eval_edge_features_2 = torch.tensor([
    [0.3, 0.1],  # edge 0 -> 1
    [-0.4, 0.2], # edge 1 -> 2
    [0.15, -0.2],# edge 2 -> 0
], dtype=torch.float)

# Edge index for evaluation (same as training, but can be different)
eval_edge_index_2 = torch.tensor([
    [0, 1, 2],
    [1, 2, 0],
], dtype=torch.long)


eval_graphs = [Data(x=eval_edge_features_1, edge_index=eval_edge_index_1),
               Data(x=eval_edge_features_2, edge_index=eval_edge_index_2)
               ]

eval_data = Batch.from_data_list(eval_graphs)



In [9]:
# Set the model to evaluation mode
model.eval()

# Make predictions using the evaluation data
with torch.no_grad():
    predictions = model(eval_data.x, eval_data.edge_index)

# Convert the predictions to probabilities using the softmax function
probs = F.softmax(predictions, dim=1)

# Get the class with the highest probability
predicted_labels = torch.argmax(probs, dim=1)

# Print the Predicted Labels
print(predicted_labels)


tensor([1, 0, 1, 1, 0, 1])
